# W&B Enterprise Workshop: Advanced Geological AI

Welcome to the W&B Enterprise Workshop. In this session, we will demonstrate how Weights & Biases serves as the indispensable **system of record** for a complex, enterprise-grade machine learning workflow. By establishing a centralized hub for all our activities, we can break down silos between geoscientists, ML engineers, and stakeholders, creating a single source of truth for the entire project lifecycle.

We will simulate the training of a **conditional diffusion model** for geological structure generation. This allows us to focus on the MLOps challenges—collaboration, monitoring, governance, and reporting—that W&B is designed to solve, without waiting hours for a real model to train.

Our first step is to install the required libraries and import our dependencies.

In [21]:
# Install minimal dependencies for W&B workshop
# The wandb-workspaces library is key for both programmatic workspaces and reporting
# allowing us to automatically generate stakeholder-ready W&B Reports later
%pip install wandb numpy tqdm wandb-workspaces plotly pillow scikit-image pyvista ipyvolume ipython-genutils trame trame-vuetify trame-vtk -q

import numpy as np
import wandb
import wandb_workspaces.reports.v2 as wr
from tqdm import tqdm
import time
import os
from pathlib import Path
from typing import List, Tuple, Dict, Any
import json
import plotly.graph_objects as go
import plotly.io as pio
from skimage.metrics import structural_similarity as ssim
from plotly.subplots import make_subplots
import pyvista as pv
import ipyvolume as ipv
from skimage.transform import resize
from textwrap import dedent

# Set PyVista to use an off-screen plotter for notebook environments
pv.set_jupyter_backend(None)




[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## 1. W&B Project Configuration

Here, we'll log in to W&B and define the `ENTITY` (your team or organization) and the `PROJECT` for this workshop. 

A W&B Project is a collaborative workspace where your entire team—from geoscientists to ML engineers—can track experiments, compare results, and share insights in real-time.

Centralizing our work here is the first step toward building a reliable system of record. We also define a sample configuration dictionary that holds our model's hyperparameters.

In [14]:

ENTITY = "wandb_emea"  # Replace with your W&B team entity
PROJECT = "workshop-scratchpad4"

# Ensure you have logged in and defined ENTITY and PROJECT in a previous cell
assert "ENTITY" in locals(), "Please define the ENTITY variable"
assert "PROJECT" in locals(), "Please define the PROJECT variable"

# W&B Team Authentication
wandb.login()


print(f"W&B Authentication: Entity = {ENTITY}")
print(f"Project: {PROJECT}")

# Configuration for (simulated) conditional diffusion training
# wandb.config makes hyperparameters a first-class citizen.
# They are saved with every run, ensuring 100% reproducibility and
# enabling powerful, automated hyperparameter sweeps

example_config = {
    # Model Architecture
    "model_architecture": "ConditionalUNet3D",
    "task": "geological_structure_generation",
    "input_modality": "seismic_amplitude",
    "output_modality": "karst_structures",
    
    # Training Parameters
    "epochs": 10,
    "batches_per_epoch": 20,
    "learning_rate": 1e-3,
    "batch_size": 4,
    
    # Diffusion Parameters
    "timesteps": 1000,
    "noise_schedule": "cosine",
    "conditioning_strength": 0.8,
    
    # Geological Domain
    "voxel_resolution": "25m",
    "depth_range": "0-800m",
    "geological_context": "karst_detection"
}

print("Training configuration loaded")

W&B Authentication: Entity = wandb_emea
Project: workshop-scratchpad4
Training configuration loaded


## 2. Simulation and Visualization Helpers

This cell contains the helper functions for our workshop. **This is not part of the core W&B integration.**

- `simulate_...`: A function that mimics the behavior of a real conditional diffusion model, generating progressively better geological predictions with each epoch.
- `plot_...` & `normalize_...`: Utilities for creating interactive charts and preparing images for visualization.

We're using a simulation because these generative models can run for days or weeks. By focusing on the MLOps workflow, we demonstrate how to solve the operational challenges—like monitoring, debugging, and reporting—where teams lose the most time and money, especially when long-running jobs fail silently.

By isolating this simulation logic, we can focus the rest of the notebook purely on the MLOps workflow powered by W&B.

In [4]:
# By returning a wandb.Html object, we can log custom, interactive visualizations
# like this Plotly chart directly into the W&B dashboard. This allows domain experts
# to analyze results without switching contexts or downloading files.
def plot_well_log_comparison(gt_log: np.ndarray, pred_log: np.ndarray, depth: np.ndarray) -> wandb.Html:
    """Creates an interactive Plotly chart comparing ground truth and predicted well logs."""
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=gt_log, y=depth, mode='lines', name='Ground Truth Log', line=dict(color='black')))
    fig.add_trace(go.Scatter(x=pred_log, y=depth, mode='lines', name='Predicted Log', line=dict(color='crimson', dash='dash')))
    fig.update_layout(
        title="Well Log Comparison",
        xaxis_title="Signal Amplitude",
        yaxis_title="Depth (m)",
        yaxis_autorange='reversed' # Depth increases downwards
    )
    html = pio.to_html(fig)
    return wandb.Html(html)

def generate_synthetic_loss(
    step: int,
    schedule: str = "cosine",
    conditioning_strength: float = 0.8,
    learning_rate: float = 1e-3,
    batch_size: int = 4,
    base_seed: int = 42,
) -> float:
    """
    Parameterized synthetic loss that responds to sweep knobs.
    - schedule: shapes the decay (cosine|linear|sigmoid)
    - conditioning_strength: higher => faster improvement
    - learning_rate: modestly speeds decay
    - batch_size: larger => lower noise
    Deterministic per step via base_seed + step.
    """
    # Progress in [0, 1]
    progress = np.clip(step / 600.0, 0.0, 1.0)

    if schedule == "linear":
        sched = progress
    elif schedule == "sigmoid":
        sched = 1.0 / (1.0 + np.exp(-10.0 * (progress - 0.5)))
    else:  # cosine
        sched = 0.5 - 0.5 * np.cos(np.pi * progress)

    # Faster decay with stronger conditioning and slightly higher LR
    decay_speed = 1.0 + 0.8 * conditioning_strength + 0.3 * np.log10(max(learning_rate, 1e-6) / 1e-3 + 1.0)
    base_curve = 1.5 * np.exp(-3.0 * sched * decay_speed) + 0.05

    # Noise shrinks with batch size
    rng = np.random.RandomState(int(base_seed) + int(step))
    noise = rng.normal(0.0, 0.05 / max(batch_size, 1))

    return float(max(0.005, base_curve + noise))

# Simulation controls: 
# conditioning_strength controls structure emergence
# noise_schedule (‘cosine’, ‘linear’, ‘sigmoid’) controls noise decay shape
# Runs are deterministic per epoch via base_seed
def simulate_conditional_diffusion_progress(
    seismic_condition: np.ndarray,
    karst_target: np.ndarray,
    epoch: int,
    total_epochs: int = 10,
    conditioning_strength: float = 0.8,
    noise_schedule: str = "cosine",
    base_seed: int = 42,
) -> np.ndarray:
    """
    Simulate conditional diffusion learning progression (single-channel).
    - conditioning_strength: scales structure emergence (higher = stronger alignment to karst_target)
    - noise_schedule: 'cosine' | 'linear' | 'sigmoid' controls noise decay shape over epochs
    - Deterministic per epoch via base_seed + epoch
    """
    # Training progress (0.0 at start, 1.0 at end) → used to shape schedule
    progress = epoch / float(max(1, total_epochs - 1))

    # Deterministic seed derived from base_seed and epoch
    np.random.seed(int(base_seed) + int(epoch) * 1000)

    # Schedule shaping (affects how fast noise decreases / structure increases)
    if noise_schedule == "linear":
        schedule_factor = progress
    elif noise_schedule == "sigmoid":
        schedule_factor = 1.0 / (1.0 + np.exp(-12.0 * (progress - 0.5)))
    else:  # "cosine" default
        schedule_factor = 0.5 - 0.5 * np.cos(np.pi * progress)

    # Stage 1 (epochs 0–3): learn basic correlations; high noise
    if epoch <= 3:
        noise_level = 0.8 - (epoch / 3.0) * 0.4          # 0.8 → 0.4
        noise_level *= (1.0 - 0.25 * schedule_factor)    # slightly faster decay with schedule
        structural_learning = (epoch / 3.0) * 0.3        # 0.0 → 0.3
        structural_learning *= conditioning_strength

        structure_mask = (seismic_condition > np.percentile(seismic_condition, 60)).astype(float)
        noise = np.random.normal(0, noise_level, seismic_condition.shape)

        prediction = (
            noise * 0.7 +
            seismic_condition * structural_learning +
            karst_target * structure_mask * 0.1
        )

    # Stage 2 (epochs 4–6): refine geological structures; reduce noise
    elif epoch <= 6:
        stage_progress = (epoch - 4.0) / 2.0             # 0.0 → 1.0
        karst_regions = (
            (seismic_condition > np.percentile(seismic_condition, 40)) &
            (seismic_condition < np.percentile(seismic_condition, 85))
        )

        noise_level = 0.4 - stage_progress * 0.25        # 0.4 → 0.15
        noise_level *= (1.0 - 0.25 * schedule_factor)
        noise = np.random.normal(0, noise_level, seismic_condition.shape)

        geological_understanding = 0.3 + stage_progress * 0.4   # 0.3 → 0.7
        geological_understanding *= conditioning_strength

        prediction = (
            noise * 0.3 +
            karst_target * karst_regions * geological_understanding +
            seismic_condition * 0.2 * (1 - karst_regions)
        )

    # Stage 3 (epochs 7–9): fine-tuning; high accuracy, low noise
    else:
        stage_progress = (epoch - 7.0) / 2.0             # 0.0 → 1.0
        noise_level = 0.15 - stage_progress * 0.10       # 0.15 → 0.05
        noise_level *= (1.0 - 0.25 * schedule_factor)
        noise = np.random.normal(0, noise_level, seismic_condition.shape)

        accuracy = 0.7 + stage_progress * 0.25           # 0.7 → 0.95
        accuracy *= (0.85 + 0.3 * conditioning_strength)
        accuracy = np.clip(accuracy, 0.0, 1.0)

        uncertainty_mask = np.random.random(seismic_condition.shape) < 0.1

        prediction = (
            karst_target * accuracy +
            noise * 0.1 +
            seismic_condition * uncertainty_mask * 0.05
        )

    # Keep realistic value ranges and avoid exact zeros in residuals
    prediction = np.clip(prediction, 0, 1)
    prediction += np.random.uniform(-1e-9, 1e-9, prediction.shape)
    return prediction

def normalize_for_visualization(data: np.ndarray) -> np.ndarray:
    """Normalize geological data for W&B visualization with proper contrast."""
    data_min, data_max = np.min(data), np.max(data)
    if data_max == data_min:
        return np.full_like(data, 0.5)
    normalized = (data - data_min) / (data_max - data_min)
    # Apply gamma correction for better geological feature visibility
    return np.power(normalized, 0.7)

def simulate_forward_model(x_pred: np.ndarray) -> np.ndarray:
    """
    Simulates a forward model y=f(x) to generate a predicted condition y_pred.
    This step is crucial for evaluating the "cycle consistency" of our generator;
    we check if the forward model of our prediction (y_pred) matches the original input condition (y).
    In a real scenario, this would be a physics-based or learned model.
    Here, we simulate it by applying a slight blur and adding minor noise.
    """
    # Apply a simple blurring effect (convolution with a small kernel)
    kernel = np.ones((3, 3, 3)) / 27.0
    # Use scipy for convolution if available, otherwise a simpler method
    try:
        from scipy.ndimage import convolve
        blurred = convolve(x_pred, kernel, mode='reflect')
    except ImportError:
        # Fallback if scipy is not installed
        blurred = x_pred
        
    # Add a small amount of random noise
    noise = np.random.normal(0, 0.02, x_pred.shape)
    y_pred = blurred + noise
    
    return np.clip(y_pred, 0, 1)

### Tier 1 Analysis: Global Sanity Check (Downsampled View)

**The Question:** "Is the model globally stable?" Before wasting time on details, we need a quick, high-level check of the entire volume.

**The Technique:** We downsample the high-resolution volumes (e.g., 128³) to a smaller size (64³) and render them side-by-side. This provides an immediate, low-latency overview.

**What We Look For:**
* **Large-Scale Artifacts:** Are there obvious blocky patterns or spatial biases where the model is systematically failing?
* **Structural Coherence:** Does the overall shape of the prediction roughly match the ground truth?
* **Performance:** This view renders instantly in the browser, providing the fastest possible feedback loop during training.

**The Limitation:**
* **Loss of Detail:** This view is for macro-level assessment only. Fine-grained geological textures and sharp boundaries are intentionally sacrificed for speed and context.

In [5]:
def create_downsampled_comparison(volumes: Dict[str, np.ndarray]) -> go.Figure:
    """Downsamples volumes and returns a side-by-side Plotly figure."""
    target_shape = (64, 64, 64)
    subplot_titles = [name.replace("_", " ") for name in volumes.keys()]
    fig = make_subplots(
        rows=1, cols=len(volumes),
        specs=[[{'type': 'volume'}] * len(volumes)],
        subplot_titles=subplot_titles
    )
    for i, (name, data) in enumerate(volumes.items()):
        resized_data = resize(data, target_shape, anti_aliasing=True)
        vmin, vmax = float(np.min(resized_data)), float(np.max(resized_data))
        if vmax == vmin: vmax = vmin + 1.0
        fig.add_trace(
            go.Volume(
                value=resized_data,                 # pass 3D array directly
                cmin=vmin, cmax=vmax,
                isomin=vmin + (vmax - vmin) * 0.02,
                isomax=vmax - (vmax - vmin) * 0.02,
                opacity=0.15, opacityscale="uniform",
                surface_count=12,
                colorscale='RdBu' if ('Residual' in name or 'Error' in name) else 'viridis',
            ),
            row=1, col=i + 1
        )
    fig.update_layout(title_text="Downsampled 3D Comparison", height=420, margin=dict(t=50, b=10, l=10, r=10))
    return fig


### Tier 2 Analysis: Fine-Detail Inspection (Cropped View)

**The Question:** "Now that the global structure looks right, is the model accurately preserving fine details?"

**The Technique:** We extract a smaller cube (e.g., 64³) from the center of the original, full-resolution volumes. This allows for a direct, apples-to-apples comparison of a specific region without any downsampling.

**What We Look For:**
* **Texture and Sharpness:** Is the model generating geologically plausible textures, or is the output blurry and overly smooth? Are the boundaries between different rock types sharp and well-defined?
* **Small-Scale Accuracy:** This is crucial for validating that the model isn't just learning a low-resolution approximation but is capturing the high-frequency details essential for interpretation.

**The Limitation:**
* **Loss of Context:** This view provides no information about performance outside the cropped region. An issue might seem small here but could be part of a much larger problem.

In [6]:
def create_cropped_comparison(volumes: Dict[str, np.ndarray]) -> go.Figure:
    """Crops the center of volumes and returns a side-by-side Plotly figure."""
    crop_size = 64
    subplot_titles = [name.replace("_", " ") for name in volumes.keys()]
    fig = make_subplots(
        rows=1, cols=len(volumes),
        specs=[[{'type': 'volume'}] * len(volumes)],
        subplot_titles=subplot_titles
    )
    for i, (name, data) in enumerate(volumes.items()):
        if all(dim >= crop_size for dim in data.shape):
            center = [dim // 2 for dim in data.shape]
            start = [c - crop_size // 2 for c in center]
            cropped_data = data[
                start[0]:start[0]+crop_size,
                start[1]:start[1]+crop_size,
                start[2]:start[2]+crop_size
            ]
        else:
            cropped_data = data
        vmin, vmax = float(np.min(cropped_data)), float(np.max(cropped_data))
        if vmax == vmin: vmax = vmin + 1.0
        fig.add_trace(
            go.Volume(
                value=cropped_data,                # pass 3D array directly
                cmin=vmin, cmax=vmax,
                isomin=vmin + (vmax - vmin) * 0.02,
                isomax=vmax - (vmax - vmin) * 0.02,
                opacity=0.15, opacityscale="uniform",
                surface_count=12,
                colorscale='RdBu' if ('Residual' in name or 'Error' in name) else 'viridis',
            ),
            row=1, col=i + 1
        )
    fig.update_layout(title_text="Cropped (Full-Res) 3D Comparison", height=420, margin=dict(t=50, b=10, l=10, r=10))
    return fig

### Tier 3 Analysis: Debugging with the Residual (High-Fidelity Render)

**The Question:** "Where, specifically, is my model failing?" To improve the model, we must understand the nature and location of its errors.

**The Technique:** We use a high-performance library like Ipyvolume or PyVista to create the best possible render of the most critical volume: **the residual** (`AI_Condition_Y_pred - Input_Condition_Y`). This shows us the error map in 3D.

**What We Look For:**
* **Systematic Errors:** Are the largest errors concentrated in a specific geological layer or region? This could indicate a need for more training data from that area.
* **Nature of the Error:** Is the model over-predicting (large positive residual) or under-predicting (large negative residual)? Advanced rendering controls (lighting, opacity) allow us to peel back the volume layer-by-layer to diagnose the problem. This is the view a domain expert would use to provide targeted feedback.

**The Trade-off:**
* **Single-Volume Focus:** This is a deep-dive tool, best used to inspect one critical volume at a time.

In [7]:
def create_pyvista_render(volume: np.ndarray, title: str) -> str:
    """
    Render a single volume with PyVista and return HTML.
    - Downsamples large volumes to speed export
    - Uses ImageData; compatible with older PyVista versions
    """
    html_filename = f"{title.replace(' ', '_').replace('(', '').replace(')', '')}_temp.html"
    try:
        # Downsample aggressively if needed (keep <= 96^3)
        max_dim = 96
        dz, dy, dx = map(int, volume.shape)
        scale = max(1, max(dz, dy, dx) // max_dim)
        target = (max(1, dz // scale), max(1, dy // scale), max(1, dx // scale))
        vol_ds = volume if (dz, dy, dx) == target else resize(volume, target, anti_aliasing=True)
        vol_ds = np.asarray(vol_ds, dtype=np.float32, order="F")

        # Build an ImageData grid (works across PyVista versions)
        nz, ny, nx = vol_ds.shape  # z, y, x
        grid = pv.ImageData(dimensions=(nx, ny, nz))  # note order: x, y, z
        grid.spacing = (1.0, 1.0, 1.0)
        grid.origin = (0.0, 0.0, 0.0)
        grid.point_data["values"] = vol_ds.flatten(order="F")

        plotter = pv.Plotter(off_screen=True, notebook=True, window_size=(800, 600))
        plotter.add_volume(
            grid,
            scalars="values",
            cmap="RdBu" if ("Residual" in title or "Error" in title) else "viridis",
            opacity="sigmoid",
            shade=True,
        )
        plotter.add_axes()
        plotter.export_html(html_filename)
        html = Path(html_filename).read_text()
        plotter.close()
        return html
    except Exception as e:
        return f"<p>PyVista rendering failed: {e}</p>"
    finally:
        try:
            if os.path.exists(html_filename):
                os.remove(html_filename)
        except Exception:
            pass

## 3. The Core Training Function with W&B Integration

This function encapsulates the entire training and evaluation workflow. It serves as a blueprint for a reproducible, auditable, and transparent ML lifecycle. Each section is clearly marked to show how a specific W&B feature is used to track, version, and evaluate the model, turning a standard training script into an enterprise-ready system of record.

**Notation and channel scope**  
X: ground-truth geology (karst), X_pred: generated prediction  
Y: input seismic condition, Y_pred = f(X_pred)  
Residual = Y_pred − Y  

Single-channel visuals: This demo renders a single channel for stability and speed. Multi-channel rendering/metrics are feasible (loop over channels and log per-channel), but intentionally out of scope here.

In [22]:
def train_conditional_diffusion(config=None):
    """
    Train a conditional diffusion model for geological structure generation.
    This function is instrumented with W&B to track the entire lifecycle.
    """
    # ===================================================================
    # 1. W&B SETUP: Initialize Run and Centralize Configuration
    #    - wandb.init() starts a new run to track this experiment.
    #    - wandb.config pulls hyperparameter settings, making them available to sweeps.
    # ===================================================================
 

    # Initialize a new W&B run
    run = wandb.init(
        entity=ENTITY,
        project=PROJECT,
        config=(config or example_config), # use config if it exists, otherwise use example_config (useful for sweeps)
        # job_type and tags are powerful organizational tools that make it easy
        # to filter, group, and compare runs across a large project in the W&B UI
        job_type="training",
        tags=["conditional-diffusion"]
    )

    # Use the config from W&B (this allows sweeps to override values)
    config = wandb.config # this is a dictionary of the hyperparameters
    np.random.seed(config.get("seed", 42))  # set global seed once
    # ===================================================================
    # 2. W&B ARTIFACTS: Versioning and Loading the Dataset
    #    - run.use_artifact() declares a dependency on a specific dataset version.
    #    - This creates a lineage graph, giving us a full audit trail.
    # ===================================================================

    # ===================================================================
    # 2. W&B ARTIFACTS: Versioning and Loading the Dataset
    #    - run.use_artifact() declares a dependency on a specific dataset version.
    #    - This creates a lineage graph, giving us a full audit trail.
    # ===================================================================
    # Load CigKarst dataset from W&B Registry
    
    
    # run.use_artifact() creates a direct link to the dataset version used for this run.
    # This automatically generates a complete, visual data-to-model lineage graph,
    # which is essential for auditability and debugging.
    artifact = run.use_artifact('wandb-registry-dataset/CigKarst:v0', type='dataset')
    artifact_dir = artifact.download()

    # Load geological samples from the dataset
    geological_samples = []
    metadata_path = Path(artifact_dir) / "dataset_metadata.npz"

    if metadata_path.exists():
        metadata = np.load(metadata_path, allow_pickle=True)
        samples_data = metadata['samples'].tolist()
        for sample_info in samples_data:
            seismic_file = Path(artifact_dir) / sample_info['seismic_file']
            karst_file = Path(artifact_dir) / sample_info['karst_file']
            if seismic_file.exists() and karst_file.exists():
                seismic_data = np.load(seismic_file)
                karst_data = np.load(karst_file)
                sample = {
                    'seismic': seismic_data['patch'],
                    'karst': karst_data['patch'],
                    'sample_id': sample_info['sample_id'],
                    'coordinates': sample_info['coordinates'],
                    'source': sample_info['source_volume']
                }
                geological_samples.append(sample)
        print(f"Loaded {len(geological_samples)} real geological samples from CigKarst")
    else:
        print("Metadata file not found in artifact")
        run.finish()
        return None

    # Select a fixed set of samples for consistent validation across runs
    good_sample_ids = ["volume_0_patch_1", "volume_0_patch_3", "volume_1_patch_0"]
    fixed_samples = [s for s in geological_samples if s['sample_id'] in good_sample_ids]
    if len(fixed_samples) != len(good_sample_ids):
        raise ValueError(f"Error: Could not find all specified good samples.")
    print(f"Selected {len(fixed_samples)} pre-validated samples for tracking.")

    # Track best model
    best_validation_loss = float('inf')

    # ===================================================================
    # 3. W&B TRAINING LOOP: Logging Live Metrics
    #    - run.log() is called inside the loop to stream metrics in real-time.
    #    - This allows us to monitor model performance and system utilization live from the dashboard.
    # ===================================================================
    for epoch in range(config.get("epochs", 10)):
        print(f"Epoch {epoch}: Simulating conditional diffusion training...")
        batches_per_epoch = config.get("batches_per_epoch", 20)
        
        for batch in range(batches_per_epoch):
            step = epoch * batches_per_epoch + batch
            # Simulate training progress and calculate synthetic losses
            noise_pred_loss = generate_synthetic_loss(
            step,
            schedule=str(config.get("noise_schedule", "cosine")),
            conditioning_strength=float(config.get("conditioning_strength", 0.8)),
            learning_rate=float(config.get("learning_rate", 1e-3)),
            batch_size=int(config.get("batch_size", 4)),
            base_seed=int(config.get("seed", 42)),
            )
            reconstruction_proxy = generate_synthetic_loss(
            int(step * 1.5),
            schedule=str(config.get("noise_schedule", "cosine")),
            conditioning_strength=float(config.get("conditioning_strength", 0.8)),
            learning_rate=float(config.get("learning_rate", 1e-3)),
            batch_size=int(config.get("batch_size", 4)),
            base_seed=int(config.get("seed", 42)),
             )
            total_loss = 0.6 * noise_pred_loss + 0.4 * reconstruction_proxy

            # Prepare log dictionary for training metrics
            # Training-time proxy metrics (synthetic): reconstruction_mse approximates X_pred vs X; true condition-consistency is computed at validation.
            log_dict = {
            "train/noise_prediction_loss": noise_pred_loss,
            "train/reconstruction_mse": reconstruction_proxy,  # X_pred vs X (synthetic proxy during training)
            "train/total_loss": total_loss,
            "train/learning_rate": config.get("learning_rate", 1e-3) * (0.95 ** (epoch // 3))
            }
            
            # SINGLE wandb.log call per training step. W&B automatically captures system
            # metrics (CPU/GPU utilization, memory) alongside your custom metrics, providing
            # a holistic view to diagnose performance bottlenecks in real-time.
            run.log(log_dict)

        # ===================================================================
        # 4. W&B VALIDATION: Logging Rich Media and Tables
        #    - At the end of each epoch, we log qualitative results.
        #    - wandb.Image() for visual comparison.
        #    - wandb.Html() for custom, interactive Plotly charts.
        #    - wandb.Table() to create structured, sortable tables of predictions.
        # ===================================================================
        
        
        # wandb.Table creates a rich, interactive table in the UI. This allows for
        # sorting and filtering results, and comparing images, plots, and metrics
        # side-by-side across different runs—all within a single view.
        validation_table = wandb.Table(columns=[
        "epoch", "sample_id", "seismic_input", "ground_truth",
        "prediction", "residual_map", "well_log_comparison",
        "reconstruction_mse", "condition_consistency_mse", "ssim_score", "log_correlation"
        ])
            
        total_reconstruction_mse = 0.0
        total_condition_consistency_mse = 0.0

        # --- NEW: Create visualizations for a small BATCH of validation samples ---
        # We will create a unique key for each visualization in the log dictionary.
        visualizations_log = {}
        
        for sample in fixed_samples:
            
            # Generate the necessary volumes for this specific sample
            prediction_3d = simulate_conditional_diffusion_progress(
            sample['seismic'],
            sample['karst'],
            epoch,
            total_epochs=int(config.get("epochs", 10)),
            conditioning_strength=float(config.get("conditioning_strength", 0.8)),
            noise_schedule=str(config.get("noise_schedule", "cosine")),
            base_seed=int(config.get("seed", 42)),
            )

            y_pred_3d = simulate_forward_model(prediction_3d)
            volumes_for_viz = {
                "Input_Seismic_(Y)": sample['seismic'],
                "Ground_Truth_Karst_(X)": sample['karst'],
                "Predicted_Karst_(X_pred)": prediction_3d,
                "Forward_Model_Seismic_(Y_pred)": y_pred_3d,
                "Model_Error_(Residual)": y_pred_3d - sample['seismic']
            }
            
            #start
            # 3D: zero/mid/last only; single-channel; PyVista default; ipyvolume off by default.
            epochs_total = int(config.get("epochs", 10))
            mid_epoch = (epochs_total - 1) // 2
            should_log_3d = bool(config.get("enable_3d", True)) and (epoch in [0, mid_epoch, epochs_total - 1])

            if should_log_3d and sample['sample_id'] == fixed_samples[0]['sample_id']:
                volumes_for_viz = {
                    "Predicted_Karst_(X_pred)": prediction_3d,
                    "Forward_Model_Seismic_(Y_pred)": y_pred_3d,
                    "Model_Error_(Residual)": y_pred_3d - sample['seismic'],
                }
                ''' Removing the plotly figures for now
                ds_fig = create_downsampled_comparison({
                    "Predicted_Karst_(X_pred)": volumes_for_viz["Predicted_Karst_(X_pred)"],
                    "Forward_Model_Seismic_(Y_pred)": volumes_for_viz["Forward_Model_Seismic_(Y_pred)"],
                    "Model_Error_(Residual)": volumes_for_viz["Model_Error_(Residual)"],
                })
                visualizations_log["3D/Tier1_Downsampled"] = wandb.Plotly(ds_fig)

                cr_fig = create_cropped_comparison({
                    "Predicted_Karst_(X_pred)": volumes_for_viz["Predicted_Karst_(X_pred)"],
                    "Forward_Model_Seismic_(Y_pred)": volumes_for_viz["Forward_Model_Seismic_(Y_pred)"],
                    "Model_Error_(Residual)": volumes_for_viz["Model_Error_(Residual)"],
                })
                visualizations_log["3D/Tier2_Cropped"] = wandb.Plotly(cr_fig)
                '''
                if bool(config.get("enable_high_fidelity_3d", True)):
                    visualizations_log["Viz/PyVista_Renders/AI_Prediction"] = wandb.Html(
                        create_pyvista_render(volumes_for_viz["Predicted_Karst_(X_pred)"], "PV_Prediction")
                    )
                    visualizations_log["Viz/PyVista_Renders/Model_Error"] = wandb.Html(
                        create_pyvista_render(volumes_for_viz["Model_Error_(Residual)"], "PV_Error")
                    )

                if bool(config.get("enable_ipyvolume", False)):
                    visualizations_log["Viz/ipyvolume_Renders/AI_Prediction"] = wandb.Html(
                        create_ipyvolume_render(volumes_for_viz["Predicted_Karst_(X_pred)"], "IPV_Prediction")
                    )
                    visualizations_log["Viz/ipyvolume_Renders/Model_Error"] = wandb.Html(
                        create_ipyvolume_render(volumes_for_viz["Model_Error_(Residual)"], "IPV_Error")
                    )

            # Prepare data for logging (2D slices, logs, etc.)
            # Single-channel rendering: we use the scalar volume (or channel 0) for slices.
            # Multi-channel is feasible later by indexing and looping channels.
            slice_idx = sample['seismic'].shape[2] // 2
            seismic_slice = sample['seismic'][:, :, slice_idx]
            gt_slice = sample['karst'][:, :, slice_idx]
            pred_slice = prediction_3d[:, :, slice_idx]
            residual_slice = pred_slice - gt_slice
            max_abs_val = np.max(np.abs(residual_slice))
            residual_norm = (residual_slice + max_abs_val) / (2 * max_abs_val) if max_abs_val > 0 else np.zeros_like(residual_slice)

            # Per-epoch 2D diagnostic: compact 2x2 central-slice grid for one representative sample (single-channel).
            if sample['sample_id'] == fixed_samples[0]['sample_id']:
                grid_top = np.hstack([
                    normalize_for_visualization(seismic_slice),  # Y
                    normalize_for_visualization(gt_slice)        # X
                ])
                grid_bottom = np.hstack([
                    normalize_for_visualization(pred_slice),     # X_pred
                    residual_norm                                # Residual (already normalized to 0..1 around 0)
                ])
                grid_img = np.vstack([grid_top, grid_bottom])

                # Stable key (no epoch in the key): W&B will store history by step
                visualizations_log["Viz/diagnostics/central_slice_grid"] = wandb.Image(
                    grid_img,
                    caption="Top: Y, X | Bottom: X_pred, Residual"
                )
            
            # Metrics
            reconstruction_mse = float(np.mean((pred_slice - gt_slice) ** 2))
            condition_consistency_mse = float(np.mean((y_pred_3d - sample['seismic']) ** 2))
            total_reconstruction_mse += reconstruction_mse
            total_condition_consistency_mse += condition_consistency_mse
            ssim_score = ssim(gt_slice, pred_slice, data_range=1.0)
            
            # Well Logs
            well_log_depth = np.arange(prediction_3d.shape[0]) * 25
            well_x, well_y = prediction_3d.shape[1] // 2, prediction_3d.shape[2] // 2
            gt_well_log = sample['karst'][:, well_x, well_y]
            pred_well_log = prediction_3d[:, well_x, well_y]
            log_correlation = np.corrcoef(gt_well_log, pred_well_log)[0, 1] if np.std(gt_well_log) > 0 and np.std(pred_well_log) > 0 else 0.0
            
            # Add data to the validation table
            validation_table.add_data(
            epoch,
            sample['sample_id'],
            wandb.Image(normalize_for_visualization(seismic_slice)),
            wandb.Image(normalize_for_visualization(gt_slice)),
            wandb.Image(normalize_for_visualization(pred_slice)),
            wandb.Image(residual_norm, caption="Residual Map"),
            plot_well_log_comparison(gt_well_log, pred_well_log, well_log_depth),
            reconstruction_mse,                 # X_pred vs X (central slice)
            condition_consistency_mse,          # Y_pred vs Y (full 3D)
            ssim_score,
            log_correlation
            )

        # Log validation metrics ONCE per epoch
        # Log all epoch-level data in a single call
        avg_recon = total_reconstruction_mse / len(fixed_samples)
        avg_cond  = total_condition_consistency_mse / len(fixed_samples)

        # Validation loss tied to actual validation metrics
        val_total_loss = 0.45 * avg_recon + 0.55 * avg_cond
        epoch_log_data = {"val/total_loss": val_total_loss, "val/avg_reconstruction_mse": total_reconstruction_mse / len(fixed_samples),
        "val/avg_condition_consistency_mse": total_condition_consistency_mse / len(fixed_samples), "val_table/validation_table": validation_table}
        epoch_log_data.update(visualizations_log) # Add the dictionary of 3D views
        run.log(epoch_log_data)

        # ===================================================================
        # 5. W&B MODEL REGISTRY: Versioning Models as Artifacts
        #    - We check if the model has improved and, if so, save it.
        #    - wandb.Artifact() creates a versioned package of model files and metadata.
        #    - run.link_artifact() registers the model in the W&B Model Registry,
        #      assigning it an alias like "best" or "staging" for easy retrieval.
        # ===================================================================
        if val_total_loss < best_validation_loss:
            best_validation_loss = val_total_loss
            print(f"New best model at epoch {epoch}! Validation loss: {val_total_loss:.4f}")
            
            # Build a concise model card as markdown; this will render as the artifact/model “card”
            model_card_md = dedent(f"""\
            # Model Card: Conditional Diffusion (Simulated)

            ## Overview
            - Task: Geological structure generation (conditional diffusion)
            - Notation: X (GT karst), X_pred (generated), Y (seismic), Y_pred = f(X_pred), Residual = Y_pred − Y
            - Demo visuals: single-channel; 3D at 0/mid/last; 2D grid every epoch

            ## Data & Lineage
            - Dataset artifact: `{artifact.name}`
            - Validation samples: {len(fixed_samples)} (pre-validated)

            ## Training Summary (this version)
            - Epoch: {epoch}
            - val/total_loss: {val_total_loss:.6f}
            - val/avg_reconstruction_mse: {avg_recon:.6f}
            - val/avg_condition_consistency_mse: {avg_cond:.6f}

            ## Key Config
            - epochs={config.get('epochs')}, batches_per_epoch={config.get('batches_per_epoch')}
            - learning_rate={config.get('learning_rate')}, batch_size={config.get('batch_size')}
            - noise_schedule={config.get('noise_schedule')}, conditioning_strength={config.get('conditioning_strength')}
            - seed={config.get('seed')}

            ## Limitations
            - Simulated training and losses for reproducible demo
            - Embedded 3D is downsampled for responsiveness
            """).strip()
            
            checkpoint_artifact = wandb.Artifact(
                name="conditional-diffusion-checkpoint",
                type="model",
                description=model_card_md,  # <- model card as markdown
                metadata={
                    "epoch": epoch,
                    "validation_loss": round(val_total_loss, 4),
                    "dataset_artifact": artifact.name
                }
            )
            
            checkpoint_path = f"best_model_epoch_{epoch}.pth"
            Path(checkpoint_path).write_text(f"Best model checkpoint at epoch {epoch}")
            checkpoint_artifact.add_file(checkpoint_path)
            
            # Log the artifact and link it to the registry
            logged_artifact = run.log_artifact(checkpoint_artifact, aliases=["best"])
           
            # run.link_artifact() registers the model in the W&B Model Registry. Aliases
            # like "staging" or "production" create pointers for CI/CD systems, automating
            # the path from training to deployment.
            run.link_artifact(
                artifact=logged_artifact,
                target_path="wandb-registry-model/conditional-diffusion",
                aliases=["staging"]
            )
            os.remove(checkpoint_path)

    # ===================================================================
    # 6. W&B CLEANUP: Finalizing the Run
    #    - run.finish() marks the run as complete and uploads any remaining data.
    # ===================================================================
    run.finish()

Task was destroyed but it is pending!
task: <Task pending name='Task-61' coro=<Event.wait() running at /opt/homebrew/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


## 4. Execute a Single Training Run

Now we call our main function to execute a single, baseline training run.

**Action:** When you run this cell, click the W&B link that appears in the output. This will take you to the live dashboard where you can see all the metrics, images, and tables being logged in real-time. This is the central hub for our experiment.

What to look for in the UI:

Live Metrics: Watch the loss curves update in real-time. No need to wait for the job to finish.

System Monitoring: Check the system tab to see live CPU/GPU utilization charts, automatically captured by W&B.

Interactive Tables: At the end of each epoch, a new row will appear in the advanced_validation_table. Click on the images to expand them and hover over the charts to interact.

In [23]:
# Train with default config
# Use the example config we defined earlier 
train_conditional_diffusion()

wandb:   49 of 49 files downloaded.  


Loaded 24 real geological samples from CigKarst
Selected 3 pre-validated samples for tracking.
Epoch 0: Simulating conditional diffusion training...
New best model at epoch 0! Validation loss: 1.4434
Epoch 1: Simulating conditional diffusion training...
New best model at epoch 1! Validation loss: 1.3642
Epoch 2: Simulating conditional diffusion training...
New best model at epoch 2! Validation loss: 1.3017
Epoch 3: Simulating conditional diffusion training...
New best model at epoch 3! Validation loss: 1.2596
Epoch 4: Simulating conditional diffusion training...
Epoch 5: Simulating conditional diffusion training...
New best model at epoch 5! Validation loss: 1.2595
Epoch 6: Simulating conditional diffusion training...
New best model at epoch 6! Validation loss: 1.2571
Epoch 7: Simulating conditional diffusion training...
Epoch 8: Simulating conditional diffusion training...
Epoch 9: Simulating conditional diffusion training...


train/learning_rate,██████████▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▃▃▃▃▃▃▃▃▃▃▃▁▁
train/noise_prediction_loss,██████████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁
train/reconstruction_mse,██████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▃▃▃▃▃▃▃▂▂▂▂▂▁▁▁▁▁▁
train/total_loss,██████▇▇▇▆▆▆▆▆▅▅▅▅▅▅▅▅▅▄▄▄▄▄▄▄▃▂▂▂▂▂▂▁▁▁
val/avg_condition_consistency_mse,▇▅▃▁▂▂▂▇██
val/avg_reconstruction_mse,█▇▇▇▆▅▅▁▁▁
val/total_loss,█▅▃▁▁▁▁▅▅▅
train/learning_rate,0.00086
train/noise_prediction_loss,0.48049
train/reconstruction_mse,0.13947
train/total_loss,0.34408


## 5. Hyperparameter Sweeps: Enterprise-Scale Optimization

Running one experiment manually is not efficient. To find the optimal model, we need to explore the hyperparameter space. W&B Sweeps provide a powerful, scalable, and fully integrated way to automate this process.

Instead of writing custom loops or relying on external optimization libraries, you can define a search strategy in a simple configuration file. W&B then coordinates the search, distributing jobs to any number of agents and providing powerful visualizations to track the results in real-time.

First, we define a `sweep_config`. This configuration specifies the search strategy (Bayesian), the metric to optimize (validation loss), and the range of hyperparameters to test. We also include an early termination strategy to save compute resources by stopping underperforming runs.

In [10]:
# Early termination: realistic Hyperband settings for epoch-based training
# - Iteration == epoch (we log val/total_loss once per epoch)
# - min_iter: let every run complete at least the first 3 epochs before pruning
# - max_iter: equal to total epochs per run (6 here). If you raise epochs, raise this too
# - eta: 3 means keep roughly the top 1/3 at each bracket; 2 is gentler, 4 is more aggressive
# - Effect: poor configs get stopped around mid-training; strong configs run to completion
early_stop = {"type": "hyperband", "min_iter": 3, "max_iter": 6, "eta": 3}

sweep_config = {
    "method": "bayes",
    "metric": {"name": "val/total_loss", "goal": "minimize"},
    "parameters": {
        "learning_rate": {"distribution": "log_uniform_values", "min": 1e-4, "max": 3e-3},
        "batch_size": {"values": [4, 8]},
        "noise_schedule": {"values": ["cosine", "linear", "sigmoid"]},
        "conditioning_strength": {"distribution": "uniform", "min": 0.6, "max": 0.95},

        # Short, deterministic runs for a demo
        "epochs": {"value": 6},
        "batches_per_epoch": {"value": 12},
        "seed": {"value": 42},

        # Keep sweeps light (no 3D during sweeps)
        "enable_3d": {"value": False},
        "enable_high_fidelity_3d": {"value": False},
        "enable_ipyvolume": {"value": False},
    },
    "early_terminate": early_stop,
}
# 2. Initialize the Sweep
# This command creates the sweep controller in the W&B cloud. It acts as a
# central coordinator that agents can query for new jobs.
sweep_id = wandb.sweep(
    sweep=sweep_config,
    entity=ENTITY,
    project=PROJECT
)

print(f"✅ Sweep created successfully! Sweep ID: {sweep_id}")
print(f"🧹 View and manage your sweep here: https://wandb.ai/{ENTITY}/{PROJECT}/sweeps/{sweep_id}")

Task was destroyed but it is pending!
task: <Task pending name='Task-5' coro=<Event.wait() running at /opt/homebrew/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


Create sweep with ID: rxh40hb8
Sweep URL: https://wandb.ai/wandb_emea/workshop-scratchpad3/sweeps/rxh40hb8
✅ Sweep created successfully! Sweep ID: rxh40hb8
🧹 View and manage your sweep here: https://wandb.ai/wandb_emea/workshop-scratchpad3/sweeps/rxh40hb8


## 6. Launch a W&B Agent

Now that the sweep is initialized, we launch an agent. The agent is a simple, stateless worker that will:
1.  Ask the W&B sweep server for a set of hyperparameters.
2.  Run our `train_conditional_diffusion` function with those hyperparameters.
3.  Repeat until the sweep is finished.

This architecture is incredibly scalable. You can launch agents on your laptop, on a fleet of cloud VMs, or in a Kubernetes cluster, and they will all coordinate through the central sweep controller to work on the same optimization problem

We will start one agent to run 5 experiments for this demo.

In [11]:
# 3. Run the Sweep Agent
# This single command connects a worker to the sweep controller. The agent
# automatically fetches a configuration, executes the training function,
# and reports the results back, requiring zero manual orchestration.
wandb.agent(sweep_id, function=train_conditional_diffusion, count=5)
wandb.teardown() # if we want to do normal runs after a sweep, in the same session, we must run this.

wandb: Agent Starting Run: 00lvj4c4 with config:
wandb: 	batch_size: 8
wandb: 	batches_per_epoch: 12
wandb: 	conditioning_strength: 0.7615948697515844
wandb: 	enable_3d: False
wandb: 	enable_high_fidelity_3d: False
wandb: 	enable_ipyvolume: False
wandb: 	epochs: 6
wandb: 	learning_rate: 0.001051215348245934
wandb: 	noise_schedule: cosine
wandb: 	seed: 42
wandb: Currently logged in as: allanstevenson (wandb_emea) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   49 of 49 files downloaded.  


Loaded 24 real geological samples from CigKarst
Selected 3 pre-validated samples for tracking.
Epoch 0: Simulating conditional diffusion training...
New best model at epoch 0! Validation loss: 1.4434
Epoch 1: Simulating conditional diffusion training...
New best model at epoch 1! Validation loss: 1.3650
Epoch 2: Simulating conditional diffusion training...
New best model at epoch 2! Validation loss: 1.3008
Epoch 3: Simulating conditional diffusion training...
New best model at epoch 3! Validation loss: 1.2590
Epoch 4: Simulating conditional diffusion training...
Epoch 5: Simulating conditional diffusion training...


train/learning_rate,████████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/noise_prediction_loss,████████▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▅▅▄▅▄▄▄▃▄▃▃▃▂▃▂▂▁
train/reconstruction_mse,███████▇██▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▅▄▄▄▃▃▃▂▂▂▂▂▂▁
train/total_loss,███████████▇▇▇▇▇▇▆▆▆▆▆▅▆▅▅▅▅▄▄▃▃▃▃▃▂▂▂▂▁
val/avg_condition_consistency_mse,█▅▃▁▂▂
val/avg_reconstruction_mse,█▆▅▅▂▁
val/total_loss,█▅▃▁▁▁
train/learning_rate,0.001
train/noise_prediction_loss,1.3089
train/reconstruction_mse,1.0655
train/total_loss,1.21154


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: p58dy0ll with config:
wandb: 	batch_size: 4
wandb: 	batches_per_epoch: 12
wandb: 	conditioning_strength: 0.8744591461858962
wandb: 	enable_3d: False
wandb: 	enable_high_fidelity_3d: False
wandb: 	enable_ipyvolume: False
wandb: 	epochs: 6
wandb: 	learning_rate: 0.0006107806772819459
wandb: 	noise_schedule: linear
wandb: 	seed: 42
Task was destroyed but it is pending!
task: <Task pending name='Task-12' coro=<Event.wait() running at /opt/homebrew/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


...

train/learning_rate,████████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/noise_prediction_loss,████▇██▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▅▅▅▄▄▄▅▄▄▃▃▄▃▂▂▃▁▁
train/reconstruction_mse,█████▇▇▇██▇▇▇▇▆▇▆▆▆▆▅▅▅▅▅▄▅▄▄▄▃▂▂▂▂▂▂▁▁▁
train/total_loss,█████▇▇▇█▇▇▇▇▇▇▇▇▇▆▆▆▅▅▅▅▅▅▄▄▄▄▃▃▃▃▃▂▂▁▁
val/avg_condition_consistency_mse,█▅▃▁▂▃
val/avg_reconstruction_mse,█▇▆▇▂▁
val/total_loss,█▅▂▁▂▁
train/learning_rate,0.00113
train/noise_prediction_loss,1.2901
train/reconstruction_mse,1.02714
train/total_loss,1.18492


wandb: Agent Starting Run: 03dipu8e with config:
wandb: 	batch_size: 8
wandb: 	batches_per_epoch: 12
wandb: 	conditioning_strength: 0.8998692112735487
wandb: 	enable_3d: False
wandb: 	enable_high_fidelity_3d: False
wandb: 	enable_ipyvolume: False
wandb: 	epochs: 6
wandb: 	learning_rate: 0.0010673011001681597
wandb: 	noise_schedule: linear
wandb: 	seed: 42
Task was destroyed but it is pending!
task: <Task pending name='Task-33' coro=<Event.wait() running at /opt/homebrew/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


wandb:   49 of 49 files downloaded.  


Loaded 24 real geological samples from CigKarst
Selected 3 pre-validated samples for tracking.
Epoch 0: Simulating conditional diffusion training...
New best model at epoch 0! Validation loss: 1.4434
Epoch 1: Simulating conditional diffusion training...
New best model at epoch 1! Validation loss: 1.3533
Epoch 2: Simulating conditional diffusion training...
New best model at epoch 2! Validation loss: 1.2890
Epoch 3: Simulating conditional diffusion training...
New best model at epoch 3! Validation loss: 1.2495
Epoch 4: Simulating conditional diffusion training...
Epoch 5: Simulating conditional diffusion training...


train/learning_rate,███████████████████▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/noise_prediction_loss,███▇▇▇▆▆▆▆▆▅▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train/reconstruction_mse,███▇▇▇▇▆▆▆▆▅▅▅▅▅▄▄▄▄▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁
train/total_loss,████▇▇▇▇▇▆▆▆▆▆▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▂▁▁▁▁
val/avg_condition_consistency_mse,█▅▃▁▂▂
val/avg_reconstruction_mse,█▆▆▆▂▁
val/total_loss,█▅▂▁▂▁
train/learning_rate,0.00101
train/noise_prediction_loss,0.83672
train/reconstruction_mse,0.61658
train/total_loss,0.74867


## 7. Programmatic Executive Reports: From Model to Boardroom

The final step is to bridge the gap between technical results and business stakeholders. This is often the "last mile" problem in ML, where valuable insights get lost in translation. Manually creating reports is slow, error-prone, and creates static documents that are quickly outdated.

The W&B Report API allows us to programmatically generate dynamic, data-driven reports directly from our experiments. This transforms our experimental results from raw data into a persistent, shareable decision-making asset. This provides a transparent, repeatable, and always up-to-date summary for governance and high-stakes investment decisions

In [12]:
print("Creating a programmatic report...")
print(f"Using entity: '{ENTITY}', project: '{PROJECT}'")

import time
import wandb
import wandb_workspaces.reports.v2 as wr

try:
    # 1) Find the best finished run by val/total_loss
    api = wandb.Api()
    runs = api.runs(
        path=f"{ENTITY}/{PROJECT}",
        order="-created_at",
        filters={"tags": "conditional-diffusion", "state": "finished"},
    )
    if not runs:
        raise ValueError("No finished runs with tag 'conditional-diffusion' found.")

    best_run = sorted(
        runs, key=lambda r: r.summary.get("val/total_loss", float("inf"))
    )[0]
    best_val = best_run.summary.get("val/total_loss", None)
    best_val_str = f"{best_val:.4f}" if best_val is not None else "N/A"
    print(f"Generating report based on best run: {best_run.name} (ID: {best_run.id})")
    if best_val is not None:
        print(f"Best validation loss: {best_val_str}")

    # 2) Build the report container (full-width page)
    report = wr.Report(
        entity=ENTITY,
        project=PROJECT,
        title=f"Geological ML Model Performance - {time.strftime('%Y-%m-%d')}",
        description=f"Automated summary for the conditional diffusion model. Best run: {best_run.name}.",
        width="fluid",  # full-width page container
    )

    # Target the best run only
    runset = wr.Runset(entity=ENTITY, project=PROJECT, name=best_run.id)

    # 3) Blocks
    report.blocks = [
        wr.H1("Executive Summary: Geological Interpretation Model"),
        wr.P(
            f"The model was trained for **{best_run.config.get('epochs', 'N/A')} epochs**, "
            f"achieving a final validation loss of **{best_val_str}**. "
            f"This automated report was generated on {time.strftime('%B %d, %Y')}."
        ),

        wr.H2("Key Performance Metrics"),
        wr.P("Validation loss over time (uses Step on the X axis)."),

        # Full-width line plot (defaults X to Step)
        wr.PanelGrid(
            runsets=[runset],
            panels=[
                wr.LinePlot(
                    title="Validation Loss Over Training",
                    y=["val/total_loss"],
                    layout={"w": 24, "h": 12},  # set width on the panel
                ),
            ],
        ),

        # Small summary scalar (optional, separate row)
        wr.PanelGrid(
            runsets=[runset],
            panels=[
                wr.ScalarChart(
                    title="Best Final Validation Loss",
                    metric="val/total_loss",
                    layout={"w": 8, "h": 8},
                )
            ],
        ),

        wr.H2("Detailed Validation Analysis"),
        wr.P("Interactive table logged each epoch; single-channel slices and well-log comparison."),

        # Full-width validation table (only panel in its grid)
        wr.PanelGrid(
            runsets=[runset],
            panels=[
                wr.WeavePanelSummaryTable(
                    table_name="advanced_validation_table",
                    layout={"w": 24, "h": 20},  # full width on the panel
                )
            ],
        ),

        wr.H2("Model Governance & Next Steps"),
        wr.P(
            f"""The best model artifact from this run has been versioned and linked to the Model Registry.

- Best Model Run Link: [`{best_run.name}`](https://wandb.ai/{ENTITY}/{PROJECT}/runs/{best_run.id})
- Dataset artifact version: `CigKarst:v0`
- Model artifact name: `conditional-diffusion-checkpoint`
- Current registry alias: `staging`

Next Steps:
1. Review model performance with geological subject matter experts.
2. Promote the model from 'Staging' to 'Production' if acceptance criteria are met.
3. Plan deployment to a validation environment.
"""
        ),
    ]

    report.save()
    print("\n✅ Executive report created successfully!")
    print(f"📊 Title: {report.title}")
    print(f"🔗 URL: {report.url}")
    print("📧 Ready for stakeholder distribution.")

except Exception as e:
    print(f"\n⚠️ Report creation failed: {e}")

Task was destroyed but it is pending!
task: <Task pending name='Task-40' coro=<Event.wait() running at /opt/homebrew/Cellar/python@3.10/3.10.18/Frameworks/Python.framework/Versions/3.10/lib/python3.10/asyncio/locks.py:214> wait_for=<Future cancelled>>


Creating a programmatic report...
Using entity: 'wandb_emea', project: 'workshop-scratchpad3'
Generating report based on best run: neat-eon-9 (ID: uv0hadse)
Best validation loss: 0.1160


wandb: Saved report to: https://wandb.ai/wandb_emea/workshop-scratchpad3/reports/Geological-ML-Model-Performance---2025-08-09--VmlldzoxMzkyMzUwOQ==



✅ Executive report created successfully!
📊 Title: Geological ML Model Performance - 2025-08-09
🔗 URL: https://wandb.ai/wandb_emea/workshop-scratchpad3/reports/Geological-ML-Model-Performance---2025-08-09--VmlldzoxMzkyMzUwOQ==
📧 Ready for stakeholder distribution.
